### GRU PIPELINE

In [1]:
from sklearn.metrics import f1_score
from pytorch_lightning import Trainer
from datetime import datetime
import torch
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

import sys
sys.path.insert(0, '/Users/florianrunkel/Documents/02_Uni/04_Masterarbeit/masterthesis/')

from ml_pipe.data.database.mongodb import MongoDb
from ml_pipe.data.dataModule.dataModule import DataModule
from ml_pipe.models.gru.model import GRUModel

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def run_pipeline():
    # Datenquelle initialisieren
    mongo = MongoDb()
    datamodule = DataModule(mongo, batch_size=32)  # Größerer Batch für bessere Stabilität
    datamodule.setup()

    # Modell initialisieren
    input_size = datamodule.train_data[0][0].shape[-1]
    sequence_length = datamodule.train_data[0][0].shape[0]
    
    model = GRUModel(
        input_size=input_size,
        hidden_size=64,    # Mehr Units für komplexere Sequenzen
        num_layers=3,      # 3 GRU-Schichten
        dropout=0.3,       # Mehr Dropout gegen Overfitting
        lr=0.001,         # Kleinere Lernrate für stabileres Training
    )

    # Trainer Setup
    trainer = Trainer(
        max_epochs=10,                    # Mehr Epochen für besseres Lernen
        enable_checkpointing=True,
        logger=True,
        enable_model_summary=True,
        log_every_n_steps=10,
        accelerator="auto",
        devices="auto",
    )

    # Training
    trainer.fit(model, datamodule=datamodule)

    # Testdaten durchlaufen
    trainer.test(model, datamodule=datamodule)

    # Evaluation
    all_preds = []
    all_targets = []

    model.eval()
    model.freeze()

    for x, y in datamodule.val_dataloader():
        x = x.to(model.device)
        y = y.to(model.device)

        with torch.no_grad():
            preds = model(x)
            preds = (preds > 0.5).float()

        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(y.cpu().numpy())

    # Metriken berechnen
    f1 = f1_score(all_targets, all_preds)
    mse = mean_squared_error(all_targets, all_preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(all_targets, all_preds)
    r2 = r2_score(all_targets, all_preds)
    print(f""" Evaluationsmetriken: F1 Score: {f1:.4f} MSE: {mse:.4f} RMSE: {rmse:.4f}  MAE: {mae:.4f} R²: {r2:.4f} """)

    # Modell speichern
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_path = f"saved_models/gru_model_{timestamp}.pt"

    # Ordner anlegen (falls nicht vorhanden) und Modell speichern
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    torch.save(model.state_dict(), model_path)

    print(f"Modell gespeichert unter: {model_path}")

run_pipeline()

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

   | Name     | Type    | Params | Mode 
----------------------------------------------
0  | gru1     | GRU     | 13.2 K | train
1  | dropout1 | Dropout | 0      | train
2  | gru2     | GRU     | 25.0 K | train
3  | dropout2 | Dropout | 0      | train
4  | gru3     | GRU     | 25.0 K | train
5  | dropout3 | Dropout | 0      | train
6  | gru4     | GRU     | 25.0 K | train
7  | dropout4 | Dropout | 0      | train
8  | fc       | Linear  | 65     | train
9  | tanh     | Tanh    | 0      | train
10 | loss_fn  | MSELoss | 0      | train
----------------------------------------------
88.2 K    Trainable params
0         Non-trainable params
88.2 K    Total params
0.353     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 9: 100%|██████████| 260/260 [00:22<00:00, 11.81it/s, v_num=9, train_loss=0.301, val_loss=0.223]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 260/260 [00:22<00:00, 11.80it/s, v_num=9, train_loss=0.301, val_loss=0.223]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 112/112 [00:02<00:00, 39.73it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.21875019371509552    │
└───────────────────────────┴───────────────────────────┘

 Evaluationsmetriken: F1 Score: 0.8070 MSE: 0.3235 RMSE: 0.5688  MAE: 0.3235 R²: -0.4782 
Modell gespeichert unter: saved_models/gru_model_20250415_113500.pt


In [14]:
def predict(input_sequence, model_path="saved_models/gru_model_20250415_113500.pt"):
    """
    Macht eine Vorhersage basierend auf einer Sequenz von Positionen.
    
    Args:
        input_sequence: Liste von Positionen, jede Position ist [Dauer, Level, Branche]
        model_path: Pfad zum gespeicherten Modell
        
    Returns:
        tuple: (Wahrscheinlichkeit, Status)
    """
    # Modell mit den korrekten Hyperparametern initialisieren
    model = GRUModel(
        input_size=3,        # 3 Features: [Dauer, Level, Branche]
        hidden_size=64,      # 64 versteckte Units
        num_layers=4,        # 4 GRU-Schichten
        dropout=0.2,         # 20% Dropout
        lr=0.01             # Lernrate
    )
    
    # Modell laden
    checkpoint = torch.load(model_path)
    
    # Prüfen, ob es sich um ein Dictionary oder direkt um state_dict handelt
    if isinstance(checkpoint, dict):
        if 'model_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['model_state_dict'])
        else:
            model.load_state_dict(checkpoint)
    else:
        model.load_state_dict(checkpoint)
    
    model.eval()

    # Input vorbereiten
    input_tensor = torch.tensor(input_sequence, dtype=torch.float32).unsqueeze(0)
    
    # Vorhersage machen
    with torch.no_grad():
        pred = model(input_tensor)
    
    # Vorhersage interpretieren
    pred_value = float(pred.item())
    
    # Interpretation der Vorhersage
    if pred_value > 0.7:
        status = "sehr wahrscheinlich wechselbereit"
    elif pred_value > 0.5:
        status = "wahrscheinlich wechselbereit"
    elif pred_value > 0.3:
        status = "möglicherweise wechselbereit"
    else:
        status = "bleibt wahrscheinlich"
    
    return pred_value, status

In [20]:
# Test für SDR zu AE Wechsel
prediction_input = [
    [64, 1, 12],  # Sales Development Representative: 12 Monate, Level 1, Sales (12)   # Account Executive: 1 Monat, Level 2, Sales (12)
]

prob, status = predict(prediction_input)
print(f"Wechselwahrscheinlichkeit: {prob:.2f} → Einschätzung: {status}")

Wechselwahrscheinlichkeit: 0.51 → Einschätzung: wahrscheinlich wechselbereit


/var/folders/hw/wbg7dg7934z5jd1dv8g7rxjc0000gn/T/ipykernel_37715/1796039434.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path)
